In [ ]:
# Reinforcement learning to play Tic Tac Toe as a first player using Policy gradient and Monte Carlo 
# state - board positions, actions - next move, method - Policy Gradient and Monte Carlo 
# the agent has learned that best move for starting first is corner  
# Learned to get 3 X's in a row
# does not know to block 3 o's is some cases 
# does not know to get 3 X's in least possible moves 
# if O inbetween two X's then agent starts to select other positions that can give 3 X's
# Performace gets better with more training 

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
# +1 if agent wins, -1 if opponent wins and zero otherwise 
def game_points(matrix):
    agent_win = np.array([1,1,1])
    opponent_win = np.array([-1,-1,-1])
    if ((matrix[0:3] == agent_win).all() | (matrix[0::4] == agent_win).all() | (matrix[0::3] == agent_win).all() | (matrix[1::3] == agent_win).all()
        | (matrix[2::3] == agent_win).all() | (matrix[2:8:2] == agent_win).all() | (matrix[3:6] == agent_win).all() | (matrix[6:9] == agent_win).all()):
        return 1
    elif ((matrix[0:3] == opponent_win).all() | (matrix[0::4] == opponent_win).all() | (matrix[0::3] == opponent_win).all() | (matrix[1::3] == opponent_win).all()
        | (matrix[2::3] == opponent_win).all() | (matrix[2:8:2] == opponent_win).all() | (matrix[3:6] == opponent_win).all() | (matrix[6:9] == opponent_win).all()):
        return -1
    else:
        return 0

def First_player_selection(player,random):
    available_positions = np.where(matrix==0)[0] 
    if random == True:
        selection = np.random.choice(available_positions)
    else:
        selection = int(model_agentF2.predict(matrix.reshape(1,9))[0])
        if selection not in available_positions:
            selection = np.random.choice(available_positions)
    if player == 'agent':
        matrix[selection] = 1
    else:
        matrix[selection] =-1
    return selection

def Second_player_selection(player,random):
    available_positions = np.where(matrix==0)[0] 
    if random == True:
        selection = np.random.choice(available_positions)
    else:
        selection = np.argmax(ada1.predict(matrix.reshape(1,9)))
        if selection not in available_positions:
            selection = np.random.choice(available_positions)
    if player == 'agent':
        matrix[selection] = 1
    else:
        matrix[selection] =-1
    return selection

In [ ]:
## agent first  
from keras import Sequential
from keras.layers import Dense
model_agentF = Sequential()
model_agentF.add(Dense(1,input_shape=(9,)))
model_agentF.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [ ]:
# agent first
num_episodes = 500
for i in range(num_episodes):
    matrix = np.zeros(9)
    turn = 0 
    opponent_selections= []
    agent_selections = []
    while ((game_points(matrix) == 0) &  (turn < 8)): 
        # state before agent takes an action 
        initial_state = matrix
        # agent makes a move 
        agent_selections.append(First_player_selection('agent',True))
        turn += 1
        # state after agent takes an action 
        after_state = matrix
        # if game not over opponent makes a move 
        if ((turn <=7) and (game_points(matrix) == 0)):
            opponent_selections.append(Second_player_selection('opponent',True))
            turn += 1
        # reward of being in a state 
        ri = game_points(initial_state)
        ra = game_points(after_state)
        #target = ra
        target = (ra+ri) + 0*(model_agentF.predict(initial_state.reshape(1,9)))
        model_agentF.fit(initial_state.reshape(1,9),target , epochs=10, verbose=0)

In [ ]:
# test values of different states 
#initial_state1 = np.array([-1,-1,-1,1,1,-1,1,0,-1])
initial_state1 = np.array([1,0,0,0,1,0,-1,0,1])
initial_state2 = np.array([-1,0,0,0,-1,0,1,0,-1])
model_agentF.predict(initial_state1.reshape(1,9)), model_agentF.predict(initial_state2.reshape(1,9))

In [ ]:
# compile a model to take input the current board state and output an action 
#agent first 
from keras import Sequential
from keras.layers import Dense
model_agentF2 = Sequential()
model_agentF2.add(Dense(1,input_shape=(9,), activation = 'linear'))
model_agentF2.compile(loss='mae', optimizer='sgd', metrics=['mae'])

In [ ]:
# train model only for episodes where agent wins. Hence, the model learns and predicts the actions required to win in a given state
# agent first 
y = 0.95
eps = 0.5
decay_factor = 0.999
num_episodes = 3000
sample= 0
for i in range(num_episodes):
    matrix = np.zeros(9)
    turn = 0 
    opponent_selections= []
    agent_selections = []
    eps *= decay_factor
    initial_state = np.zeros(9)
    after_state = np.ones(9)
    input_vector = np.zeros([1,9])
    r=0
    while ((game_points(matrix) == 0) &  (turn < 8)):
        initial_state = matrix
        if (np.random.random() > eps):
            agent_selections.append(First_player_selection('agent',False))
        else:
            agent_selections.append(First_player_selection('agent',True))
        turn += 1
        after_state = matrix
        if ((turn <=7) and (game_points(after_state) == 0)):
            opponent_selections.append(Second_player_selection('opponent',True))
            turn += 1            
        # store the state and target vector to train model 
        if (game_points(after_state) == 0):
            input_vector= np.vstack([input_vector,initial_state])
    target = game_points(matrix)
    output_vector = np.array(agent_selections)[:,np.newaxis] 
    if target == 1:
        sample +=1 
        model_agentF2.fit(input_vector, output_vector, epochs=100, verbose=0)
print(sample)

In [ ]:
initial_state1 = np.array([ -1, 1,  1,  0,  1, 0,  0, 0, 0])
ada.predict(initial_state1.reshape(1,9))

In [ ]:
initial_state, after_state, input_vector, output_vector

In [ ]:
import matplotlib.pyplot as plt 
def plot_matrix(selection, agent):
    for x in range(4):
        plt.plot([x, x], [0,3], 'k')
    for y in range(4):
        plt.plot([0, 3], [y,y], 'k')
    if agent == False:
        s = plt.plot((np.mod(selection,3) + 0.5),(2.5-int(selection/3)),
                          'o',markersize=30, markeredgecolor=(1,0,0), markerfacecolor='w', markeredgewidth=2)
    else:
        s = plt.plot((np.mod(selection,3) + 0.5),(2.5-int(selection/3)),
                          'x',markersize=30, markeredgecolor=(0,1,0), markerfacecolor='w', markeredgewidth=2)

In [ ]:
%matplotlib inline
# simple greedy selection method 
matrix = np.zeros(9)
def play_game():
    turn =0
    opponent_selections= []
    agent_selections = []
    agent_first=True
    #if np.random.random() > 0.5:
        #agent_first = True
    print('agent_first', agent_first)
    while ((game_points(matrix) == 0) &  (turn < 8)): 
        
        if agent_first == True:
            agent_selections.append(First_player_selection('agent',False))
            turn +=1
            if ((turn <=7) and (game_points(matrix) == 0)):
                opponent_selections.append(Second_player_selection('opponent',True))
                turn+=1
        else:
            opponent_selections.append(First_player_selection('opponent',True))
            turn +=1
            if ((turn <=7) and (game_points(matrix) == 0)):
                agent_selections.append(Second_player_selection('agent',False))
                turn+=1
                
        plot_matrix(opponent_selections[-1],False)
        plot_matrix(agent_selections[-1],True)
    print(matrix, agent_selections,opponent_selections)
    if game_points(matrix) == 1:
        print('agent wins')
    elif game_points(matrix) == -1:
        print('opponent wins')
    else:
        print('draw')
play_game()

In [ ]:
(model_agentF2.predict(matrix.reshape(1,9))[0])

In [ ]:
model2.save('my_model_agent_second.h5')
model_agentF2.save('my_model_agent_first.h5')

In [ ]:
from keras.models import load_model
model3 = load_model('my_model.h5')

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))